In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


c:/Users/PC/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [2]:
save_dir=f"{exps_dir}/exp3/exp_smote"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp3/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp3/test_EDA.csv')
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,...,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,...,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,...,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,...,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,...,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [4]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

True

In [5]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [6]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [7]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
print(cols_to_drop)
cols_to_drop.remove('isFraud')
len(cols_to_drop)

['R_emaildomain_3', 'card3', 'P_emaildomain_3', 'c7', 'ENTITY_TYPE', 'isFraud']


5

In [8]:
train.shape, test.shape

((561013, 98), (29527, 99))

In [9]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [10]:
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,...,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,...,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,...,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,...,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,...,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [11]:
train.isnull().sum()

isFraud                 0
transactionamt          0
productcd               0
card1                   0
card2                8604
                    ...  
P_emaildomain_2     89737
P_emaildomain_3    558111
R_emaildomain_1    429350
R_emaildomain_2    429443
R_emaildomain_3    558436
Length: 98, dtype: int64

In [12]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [13]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns
categorical_cols

Index(['productcd', 'card6', 'p_emaildomain', 'r_emaildomain', 'devicetype',
       'deviceinfo', 'EVENT_ID', 'ENTITY_ID', 'EVENT_TIMESTAMP',
       'P_emaildomain_1', 'P_emaildomain_2', 'R_emaildomain_1',
       'R_emaildomain_2'],
      dtype='object')

In [14]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns

for col in continuous_cols:
    train[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    train[col].fillna(train[col].mode()[0], inplace=True)

train.isnull().sum()

isFraud               0
transactionamt        0
productcd             0
card1                 0
card2                 1
                     ..
id_02_to_std_card6    4
P_emaildomain_1       0
P_emaildomain_2       0
R_emaildomain_1       0
R_emaildomain_2       0
Length: 93, dtype: int64

In [15]:
train = train.drop(['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID'], axis=1)
test = test.drop(['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID'], axis=1)

In [16]:
train=train.dropna()

In [17]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'devicetype', 'deviceinfo', 'productcd', 'card4', 'card6', 'M4','p_emaildomain',
            'r_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

In [18]:
train.columns

Index(['isFraud', 'transactionamt', 'productcd', 'card1', 'card2', 'card5',
       'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain', 'c1', 'c2',
       'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'v62',
       'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160',
       'v165', 'v187', 'v203', 'v207', 'v209', 'v210', 'v221', 'v234', 'v257',
       'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274', 'v277', 'v283',
       'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06',
       'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo',
       'transactionamt_to_mean_card1', 'transactionamt_to_mean_card2',
       'transactionamt_to_std_card1', 'transactionamt_to_mean_card3',
       'transactionamt_to_std_card3', 'transactionamt_to_mean_card5',
       'transactionamt_to_std_card5', 'transactionamt_to_mean_card6',
       'transactionamt_to_std_card6', 'transactionamt_to_std_card2',
       'id_02_to_mean_card1'

In [19]:
X_train = train.drop(['isFraud'], axis=1)
y_train = train['isFraud']


# del train
# test = test[["TransactionDT", 'TransactionID']]

In [20]:
train

,isFraud,transactionamt,productcd,card1,card2,card5,card6,addr1,dist1,p_emaildomain,...,id_02_to_mean_card3,id_02_to_std_card3,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
10,0,75.887,0,5591,862,112,1,494,16.333333,16,...,0.845008,1.146881,0.714929,1.097381,1.198614,1.274405,15,1,15,1
11,0,16.495,0,9234,974,835,2,494,13.666667,19,...,0.864318,1.309737,0.991947,1.307256,1.173326,1.345632,17,1,17,1
12,0,50.000,4,8723,1263,924,2,494,11.000000,49,...,0.704196,1.060140,0.804241,1.056462,0.946552,1.085556,40,6,15,1
13,0,40.000,4,2502,632,924,2,1740,8.333333,2,...,0.544074,0.810543,0.616535,0.805668,0.719779,0.825480,2,1,15,1
14,0,10.500,4,1616,1759,924,2,679,5.666667,54,...,0.383951,0.560946,0.428829,0.554873,0.493005,0.565404,43,1,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561008,0,44.500,4,2217,718,924,2,2551,3.000000,16,...,0.020794,0.020156,0.017252,0.016786,0.015230,0.016193,15,1,15,1
561009,0,125.000,3,4969,1702,2,1,1740,3.000000,16,...,0.009086,0.008807,0.007538,0.007335,0.006655,0.007076,15,1,44,1
561010,0,125.000,3,4969,1702,2,1,1740,3.000000,16,...,0.029514,0.028608,0.024486,0.023824,0.021617,0.022983,15,1,44,1
561011,0,78.694,0,8817,173,835,1,1740,3.000000,16,...,0.861154,1.304943,0.988316,1.302471,1.382436,1.469851,15,1,15,1


In [21]:
y_train

10        0
11        0
12        0
13        0
14        0
         ..
561008    0
561009    0
561010    0
561011    0
561012    0
Name: isFraud, Length: 561003, dtype: int64

In [22]:
y_train.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [23]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.5179997008359972, 1: 14.38911972914743}

In [24]:
from imblearn.over_sampling import SMOTE
x_train, y_train = SMOTE().fit_resample(X_train, y_train)


In [25]:
from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, random_state = 101)
X_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)
# x_valid.to_csv(f'{save_dir}/x_valid.csv', index=False)
# y_valid.to_csv(f'{save_dir}/y_valid.csv', index=False)

In [26]:
test = test.drop(['EVENT_ID.1'], axis=1)
for col in continuous_cols:
    if col in ['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID']:
        continue
    test[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    if col in ['EVENT_ID', 'EVENT_TIMESTAMP', 'ENTITY_ID']:
        continue
    test[col].fillna(test[col].mode()[0], inplace=True)
test=test.dropna()
x_test = test.drop(['isFraud'], axis=1)
y_test = test['isFraud']

x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)